In [1]:
import pandas as pd
import os
import dgl
import torch
from scipy.sparse import csr_matrix, coo_matrix 
import numpy as np
import dgl.function as fn
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import dgl.function as fn
import dgl.nn.pytorch as dglnn
import time
import argparse
from dgl.data import *
import tqdm
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

ImportError: No module named 'dgl'

In [2]:
print(os.getcwd())
os.chdir('/home/dqw_zyf/DTI/data')

/home/dqw_zyf/GraphDTI/code


In [3]:
alldata = pd.read_csv('alldata.csv') #491718
tar_embeddings = pd.read_csv('corpus2_tar_embeddings.csv')
mol_embeddings = pd.read_csv('corpus2_mol_embeddings.csv')

alldata.drop_duplicates(inplace = True)
alldata = alldata[(alldata.notna())] #462361

In [5]:
len(alldata)

462361

In [6]:
data = pd.merge(alldata,tar_embeddings)
print(len(data))
data1 = pd.merge(data,mol_embeddings)
print(len(data1))
data1.drop_duplicates(inplace = True)
data1 = data1[(data1.notna())] #462361
print(len(data1))

462361
462361
462361


In [7]:
len(data1['mol'].unique())#281876

281876

In [8]:
len(data1['tar_id'].unique()) #5099

5099

In [9]:

#转变成边分类任务
#index = data1['labels'] #239903  
index_np = np.array(data1[['mol_id','tar_id']])
index_dict = list(map(tuple, index_np))

reindex_np =  np.array(data1[['tar_id','mol_id']])
reindex_dict = list(map(tuple, reindex_np))
##生成图
g = dgl.heterograph({  ('mol', 'links', 'tar'): (index_dict),
                         ('tar', 'links-by', 'mol'): (reindex_dict)                                     } )

#graph(num_nodes={'mol': 281876, 'tar': 5099},
#      num_edges={('mol', 'links', 'tar'): 462361},
#      metagraph=[('mol', 'tar', 'links')])

In [10]:
g

Graph(num_nodes={'mol': 281876, 'tar': 5099},
      num_edges={('mol', 'links', 'tar'): 462361, ('tar', 'links-by', 'mol'): 462361},
      metagraph=[('mol', 'tar', 'links'), ('tar', 'mol', 'links-by')])

In [11]:

#节点特征

mol_feats = np.array(mol_embeddings.iloc[:,1:101])
mol_feats = list(map(tuple, mol_feats))

tar_feats = np.array(tar_embeddings.iloc[:,1:101])
tar_feats = list(map(tuple,tar_feats))

g.nodes['mol'].data['feats'] = torch.Tensor(mol_feats)
g.nodes['tar'].data['feats'] = torch.Tensor(tar_feats)


##边的标签
g.edges['links'].data['label'] = torch.Tensor(np.array(data1['labels']))


# 进行训练、验证和测试集划分
train_index = data1[data1['Curation/DataSource']=='ChEMBL'].index ##399553
val_index = data1[data1['Curation/DataSource']=='Curated from the literature by BindingDB'].index ##4022
test_index =  data1[data1['Curation/DataSource']=='PubChem'].index

train_mask = np.zeros((462361),dtype = int)

for i in train_index:
    train_mask[i] = 1

val_mask = np.zeros((462361),dtype = int)
for i in val_index:
    val_mask[i] = 1

test_mask = np.zeros((462361),dtype = int)
for i in test_index:
    test_mask[i] = 1    

g.edges['links'].data['train_mask'] = torch.BoolTensor(np.asarray(train_mask).astype(bool))
g.edges['links'].data['val_mask'] = torch.BoolTensor(np.asarray(val_mask).astype(bool))
g.edges['links'].data['test_mask'] = torch.BoolTensor(np.asarray(test_mask).astype(bool))

In [13]:
# Define a Heterograph Conv model

class RGCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, rel_names):
        super().__init__()
        # 实例化HeteroGraphConv，in_feats是输入特征的维度，out_feats是输出特征的维度，aggregate是聚合函数的类型
        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(in_feats, hid_feats)
            for rel in rel_names}, aggregate='sum')
        self.conv2 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(hid_feats, out_feats)
            for rel in rel_names}, aggregate='sum')

    def forward(self, graph, inputs):
        # 输入是节点的特征字典
        h = self.conv1(graph, inputs)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h

class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, rel_names):
        super().__init__()
        self.sage = RGCN(in_features, hidden_features, out_features, rel_names)
        self.pred = HeteroDotProductPredictor()
    def forward(self, g, x, etype):
        h = self.sage(g, x)
        return self.pred(g, h, etype)

class HeteroDotProductPredictor(nn.Module):
    def forward(self, graph, h, etype):
        # h是从5.1节中对每种类型的边所计算的节点表示
        with graph.local_scope():
            graph.ndata['h'] = h   #一次性为所有节点类型的 'h'赋值
            graph.apply_edges(fn.u_dot_v('h', 'h', 'score'), etype=etype)
            return graph.edges[etype].data['score']

In [14]:
model = Model(100, 20, 2, g.etypes)
mol_feats = g.nodes['mol'].data['feats']
tar_feats = g.nodes['tar'].data['feats']
label = g.edges['links'].data['label']
train_mask = g.edges['links'].data['train_mask']
node_features = {'mol': mol_feats, 'tar': tar_feats}

In [18]:
label[train_mask]

tensor([1., 0., 1.,  ..., 0., 1., 1.])

In [41]:
opt = torch.optim.Adam(model.parameters())
for epoch in range(1000):
    logits = model(g, node_features, 'links')
    pred =  pred.squeeze()
  
    #loss_fcn = nn.CrossEntropyLoss()
    #loss = torch.nn.BCEWithLogitsLoss(reduction='sum')(pred[train_mask], label[train_mask])
    #acc = (round(pred) == labels).float().sum() / len(pred)
    #loss = loss_fcn(pred[train_mask],label[train_mask])
    #loss = (abs(pred[train_mask] - label[train_mask])).mean()
    opt.zero_grad()
    loss.backward()
    opt.step()
    print(loss.item(),acc.item())

TypeError: type Tensor doesn't define __round__ method

In [46]:
class HeteroMLPPredictor(nn.Module):
    def __init__(self, in_dims, n_classes):
        super().__init__()
        self.W = nn.Linear(in_dims * 2, n_classes)

    def apply_edges(self, edges):
        x = torch.cat([edges.src['h'], edges.dst['h']], 1)
        y = self.W(x)
        return {'score': y}

    def forward(self, graph, h):
        # h是从5.1节中对异构图的每种类型的边所计算的节点表示
        with graph.local_scope():
            graph.ndata['h'] = h   #一次性为所有节点类型的 'h'赋值
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']

In [47]:
class Model2(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, rel_names):
        super().__init__()
        self.sage = RGCN(in_features, hidden_features, out_features, rel_names)
        self.pred = HeteroMLPPredictor(out_features, len(rel_names))
    def forward(self, g, x, dec_graph):
        h = self.sage(g, x)
        return self.pred(dec_graph, h)

In [49]:
dec_graph = g['mol', :, 'tar']

In [54]:
g.etypes

['links', 'links-by']

In [55]:
dec_graph

Graph(num_nodes={'mol': 281876, 'tar': 5099},
      num_edges={('mol', 'links', 'tar'): 462361},
      metagraph=[('mol', 'tar', 'links')])

In [81]:
model2 = Model(100, 200, 2, g.etypes)
opt = torch.optim.Adam(model.parameters())
for epoch in range(1000):
    logits = model2(g, node_features,'links')
    #acc = (round(pred) == labels).float().sum() / len(logits)
    #float_logits = logits.float()
    #float_label = label.long()
    #loss = nn.CrossEntropyLoss()
    #crossentropyloss=nn.CrossEntropyLoss()
    #crossentropyloss_output=crossentropyloss(logits, label)
    logits =  logits.squeeze()
    loss = torch.nn.BCEWithLogitsLoss(reduction='sum')(logits[train_mask], label[train_mask])
    opt.zero_grad()
    loss.backward()
    opt.step()
    print(loss.item())

276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625
276374.0625


KeyboardInterrupt: 

In [80]:
logits

tensor([-0.0710, -0.1623, -0.1503,  ..., -0.1170, -0.0886, -0.0886],
       grad_fn=<SqueezeBackward0>)

In [73]:
 float_label

tensor([1, 1, 0,  ..., 0, 1, 1])

In [66]:
logits.astype(FloatTensor)

AttributeError: 'Tensor' object has no attribute 'astype'

In [59]:
label

tensor([1., 1., 0.,  ..., 0., 1., 1.])